# convert avi → dataset for machine learning

In [4]:
from pathlib import Path
import numpy as np

import shutil

In [2]:
# avi → images
# from https://note.nkmk.me/python-opencv-video-to-still-image/

import cv2
import os

def save_all_frames(video_path, dir_path, basename, ext='jpg'):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    n = 0

    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
            n += 1
        else:
            return


In [3]:
# separate into frames and name as index number from 0
frame_output_dir = '../data/frames'
save_all_frames('/path/to/your/output_final_longer.avi', frame_output_dir, 'frame')


In [5]:
# check movie's fps
fps = int(19266/(17*60+50))
fps

18

In [6]:
# calc frame per minute
fpm = fps * 60
fpm

1080

In [7]:
# calc image frame index when an attack against CCTV begins,
# which can be used as train data for a machine learning model

attack_begin_frame_idx = fpm * 5
attack_begin_frame_idx

5400

In [8]:
# here is index which represents begin of test for machine learning model
crash_frame_idx = 5454
end_frame = 19266

In [20]:
# I check flagman occurence start and end time in 5 min
appear_about_sec_span = [
    [9,16],  # 'T'
    [56,60*1+7],  # 'M'
    [60*1+56, 60*2+6],  # 'C'
    [60*2+38,60*2+46],  # 'T'
    [60*3+20, 60*3+35],  # 'F'
    [60*4+31, 60*4+40]  # "{"
]

for elem in appear_about_sec_span:
    print(np.array(elem) * fps)

[162 288]
[1008 1206]
[2088 2268]
[2844 2988]
[3600 3870]
[4878 5040]


In [21]:
train_frame = [
    [132, 297],
    [1016, 1198],
    [2072, 2272],
    [2826, 2989],
    [3605, 3870],
    [4887, 5032]
    
]

In [9]:
# 修正
train_frame = [
    [148, 284],
    [1016, 1180],
    [2102, 2262],
    [2852, 2981],
    [3605, 3835],
    [4894, 5025]
    
]

In [24]:
str(5).zfill(5)

'00005'

In [10]:
# train(pos, neg), eval

pos_list = []
neg_list = []

label_list = []

r_count = 0
for x in range(0, attack_begin_frame_idx):
    x_org = x
    x = str(x).zfill(5)
    if r_count >= len(train_frame):
        neg_list.append(x)
        label_list.append([x,0])
        continue
    
    if train_frame[r_count][0] <= x_org < train_frame[r_count][1]:
        pos_list.append(x)
        label_list.append([x,1])
    elif x_org <= train_frame[r_count][0]:
        neg_list.append(x)
        label_list.append([x,0])
    elif x_org == train_frame[r_count][1]:
        r_count += 1
        print(r_count)
        neg_list.append(x)
        label_list.append([x,0])
    else:
        neg_list.append(x)
        label_list.append([x,0])



1
2
3
4
5
6


In [31]:
len(pos_list), pos_list[0]

(1120, '00132')

In [30]:
len(neg_list)

4280

In [12]:
# make labelfile
with open('../data/train_re.txt', 'wt') as f:
    for record in label_list:
        record = [record[0], str(record[1])]
        s = ','.join(record) + '\n'
        f.write(s)